In [1]:
import pandas as pd
import os

# New working directory path
new_directory = 'D:/学习/5206/pre/data'

# Set the working directory
os.chdir(new_directory)

# Get the current working directory to check if it has been successfully set
current_directory = os.getcwd()

print("Current working directory:", current_directory)

Current working directory: D:\学习\5206\pre\data


In [2]:
#pip install openpyxl

# 获取工作路径下的所有文件
file_list = [f for f in os.listdir(current_directory) if f.endswith('.xlsx')]

print(file_list)

# 遍历文件列表，读取Excel文件并存储为数据框
dataframes = {}
for file_name in file_list:
    # 构建数据框的名称，使用文件名作为键
    dataframe_name = os.path.splitext(file_name)[0]
    # 读取Excel文件
    file_path = os.path.join(current_directory, file_name)
    dataframes[dataframe_name] = pd.read_excel(file_path)



['closed.xlsx', 'gpt_score.xlsx', 'high.xlsx', 'low.xlsx', 'open.xlsx', 'output.xlsx', 'pre_closed.xlsx', 'pre_closed_exchange.xlsx', 'turnover.xlsx']


In [3]:
# 逐一提取数据框并将它们的名称作为变量名
for dataframe_name, dataframe in dataframes.items():
    # 将数据框的名称作为变量名，将数据框存储到全局变量中
    globals()[dataframe_name] = dataframe

现在有7个data frame：closed, high, low, open, pre_closed, pre_closed_exchange, turnover.

In [4]:
# 定义中位数去极值法的函数，并对异常值进行替换

def mad_outlier(series, constant=1.4826, threshold=3.5):
    median = series.median()  # 计算数据列的中位数
    mad = constant * abs(series - median).median()  # 计算绝对偏差的中位数（MAD）
    modified_z_score = 0.6745 * (series - median) / mad  # 计算修改后的Z分数
    
    # 替换过大的异常值为中位数加上常数乘以MAD的积
    series[modified_z_score > threshold] = median + constant * mad
    # 替换过小的异常值为中位数减去常数乘以MAD的积
    series[modified_z_score < -threshold] = median - constant * mad
    
    return series

#输出的series将是清洗好的

In [5]:
def column_clean(dataframe):
    for stock_column in dataframe.columns[1:]:  # 从第二列开始，跳过日期列
        dataframe[stock_column] = mad_outlier(dataframe[stock_column])
    
    return dataframe

closed = column_clean(closed)
high = column_clean(high)
low = column_clean(low)
pre_closed = column_clean(pre_closed)
pre_closed_exchange = column_clean(pre_closed_exchange)
open = column_clean(open)
turnover = column_clean(turnover)

#经过上述操作后，数据已经成功去极值化。

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16068\699176993.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[modified_z_score > threshold] = median + constant * mad
C:\Users\lenovo\AppData\Local\Temp\ipykernel_16068\699176993.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[modified_z_score < -threshold] = median - constant * mad
C:\Users\lenovo\AppData\Local\Temp\ipykernel_16068\699176993.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16068\699176993.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[modified_z_score > threshold] = median + constant * mad
C:\Users\lenovo\AppData\Local\Temp\ipykernel_16068\699176993.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[modified_z_score < -threshold] = median - constant * mad
C:\Users\lenovo\AppData\Local\Temp\ipykernel_16068\699176993.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [6]:
closed = closed.drop(columns='Date')
high = high.drop(columns='Date')
low = low.drop(columns='Date')
pre_closed = pre_closed.drop(columns='Date')
pre_closed_exchange = pre_closed_exchange.drop(columns='Date')
turnover = turnover.drop(columns='Date')
open = open.drop(columns='Date')

#去掉日期列，以免计算时被干扰。

需要考虑的是目前close可以作为价格变量，前收盘价pre_close可能有用，low和high还有open以及turnover就是可以应用的。

同时使用close可以计算当日收益率，利用open和pre_close可以计算盘前收益率。

接下来只会对数据做最后一次清洗，以及计算刚刚的两个收益率后再进行一次清洗。

清洗的步骤主要包括标准化和处理缺失值。处理缺失值我这边考虑的是标准化后可将缺失值直接设为0.

In [7]:
#计算当日收益率，log-return。

import numpy as np
import pandas as pd


# 将日期列转换为数据列
open_prices = open
pre_closed_prices = pre_closed

# 计算盘前收益率，避开缺失值
pre_market_returns = np.log(open_prices[open_prices.notna()] / pre_closed_prices[open_prices.notna()])

# 将缺失值填充为0
pre_market_returns = pre_market_returns.fillna(0)



In [8]:
print(pre_market_returns)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      0.001820   0.009647   0.010803  -0.003797   0.004442   0.011385   
1      0.004602  -0.004130  -0.003058  -0.003797  -0.003653   0.000000   
2      0.000000  -0.002099   0.000000  -0.003797  -0.004666   0.003761   
3     -0.010481  -0.008680   0.002251  -0.003797   0.000000   0.002956   
4     -0.004662   0.000000   0.001483  -0.442090  -0.009217  -0.007547   
...         ...        ...        ...        ...        ...        ...   
2300   0.006814   0.000000   0.007067   0.009662  -0.001514   0.010471   
2301   0.003454   0.002130  -0.002370  -0.004835   0.000000   0.002965   
2302  -0.008439   0.007134  -0.008470   0.003210   0.004605   0.002972   
2303   0.000000   0.000000   0.010889   0.000000   0.003086   0.000745   
2304  -0.001726   0.000000  -0.013374   0.003190  -0.004640   0.000000   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      0.000000   0.003935   0.0

In [9]:
# 将日期列转换为数据列
close_prices = closed
pre_closed_prices = pre_closed

# 计算当日收益率，避开缺失值
daily_returns = np.log(close_prices[close_prices.notna()] / pre_closed_prices[close_prices.notna()])

# 将缺失值填充为0
daily_returns = daily_returns.fillna(0)

In [10]:
print(daily_returns)

      000009.SZ  000012.SZ  000021.SZ     000027.SZ  000031.SZ  000039.SZ  \
0     -0.012832  -0.009741   0.016160 -2.220446e-16  -0.023424  -0.000764   
1     -0.026170  -0.017148   0.022642 -2.220446e-16  -0.021189   0.013657   
2     -0.000947   0.030454  -0.006739 -2.220446e-16   0.001860   0.017924   
3      0.018780  -0.054321   0.012691  4.382931e-01   0.012927  -0.015666   
4      0.047239  -0.010793  -0.001485 -4.382931e-01   0.008223   0.001503   
...         ...        ...        ...           ...        ...        ...   
2300  -0.012038   0.000000  -0.001183  6.451635e-03  -0.004549   0.012701   
2301   0.028988  -0.006417  -0.017911  0.000000e+00  -0.012233  -0.002230   
2302  -0.023811   0.009501   0.002407  0.000000e+00  -0.004626  -0.001489   
2303  -0.001723  -0.004739   0.007273  6.410278e-03   0.001544   0.000745   
2304  -0.015639  -0.011948  -0.018282 -9.630893e-03  -0.009302  -0.004671   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH

注意：当日收益率，和盘前收益率都是2305个观测值，等同于前面的7个数据框。

In [11]:
print(closed)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0         10.84      19.41      13.10   8.438387      10.97      13.09   
1         10.56      19.08      13.40   8.438387      10.74      13.27   
2         10.55      19.67      13.31   8.438387      10.76      13.51   
3         10.75      18.63      13.48  13.080000      10.90      13.30   
4         11.27      18.43      13.46   8.438387      10.99      13.32   
...         ...        ...        ...        ...        ...        ...   
2300       5.78       4.69       8.45   6.220000       6.58      13.47   
2301       5.95       4.66       8.30   6.220000       6.50      13.44   
2302       5.81       4.23       8.32   6.220000       6.47      13.42   
2303       5.80       4.21       8.28   6.260000       6.48      13.43   
2304       5.71       4.16       8.13   6.200000       6.42      10.68   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0          6.75  28.200000      

In [12]:
#标准化

from sklearn.preprocessing import StandardScaler

# 创建StandardScaler对象
scaler = StandardScaler()

# 对当日收益率数据进行标准正态化
scaled_daily_returns = scaler.fit_transform(daily_returns)

# 将标准正态化后的数据转换回数据框
scaled_daily_returns_df = pd.DataFrame(scaled_daily_returns, columns=daily_returns.columns)

# 输出标准化后的当日收益率数据框
print(scaled_daily_returns_df)


def standardize_dataframe(input_df):
    # 创建StandardScaler对象
    scaler = StandardScaler()
    
    # 使用StandardScaler对所有列进行标准正态化
    standardized_df = pd.DataFrame(scaler.fit_transform(input_df), columns=input_df.columns)
    
    return standardized_df

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0     -0.424557  -0.296807   0.572855  -0.000662  -0.620680  -0.028577   
1     -0.865962  -0.526988   0.802878  -0.000662  -0.560892   0.463238   
2     -0.031216   0.952377  -0.239758  -0.000662   0.055622   0.608762   
3      0.621671  -1.682273   0.449756  10.097694   0.351627  -0.536815   
4      1.563527  -0.329509  -0.053304 -10.099017   0.225801   0.048713   
...         ...        ...        ...        ...        ...        ...   
2300  -0.398264   0.005929  -0.042585   0.147985  -0.115811   0.430624   
2301   0.959498  -0.193503  -0.636203  -0.000662  -0.321327  -0.078574   
2302  -0.787887   0.301209   0.084791  -0.000662  -0.117875  -0.053321   
2303  -0.056872  -0.141361   0.257467   0.147032   0.047168   0.022870   
2304  -0.517438  -0.365378  -0.649373  -0.222560  -0.242953  -0.161825   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      0.430771   0.330242   0.2

In [13]:
print(closed)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0         10.84      19.41      13.10   8.438387      10.97      13.09   
1         10.56      19.08      13.40   8.438387      10.74      13.27   
2         10.55      19.67      13.31   8.438387      10.76      13.51   
3         10.75      18.63      13.48  13.080000      10.90      13.30   
4         11.27      18.43      13.46   8.438387      10.99      13.32   
...         ...        ...        ...        ...        ...        ...   
2300       5.78       4.69       8.45   6.220000       6.58      13.47   
2301       5.95       4.66       8.30   6.220000       6.50      13.44   
2302       5.81       4.23       8.32   6.220000       6.47      13.42   
2303       5.80       4.21       8.28   6.260000       6.48      13.43   
2304       5.71       4.16       8.13   6.200000       6.42      10.68   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0          6.75  28.200000      

In [14]:
#缺失值设置为0
closed = closed.fillna(0)
high = high.fillna(0)
low = low.fillna(0)
open = open.fillna(0)
pre_closed = pre_closed.fillna(0)
turnover = turnover.fillna(0)

直到现在数据已经完全清理完成。包括8个数据集，closed、high、low、open、pre_closed、turnover、daily_returns、pre_market_returns。

接下来是，计算变量。将每个股票的每个技术指标计算出来之后，我们再去计算IC、ICIR、IC均值，以及计算多空收益率之类的。

In [15]:
#例如计算第一个技术指标 ACD。以ACD指标6个交易日的收集力量与派发力量之和为例。

# 计算LC：REF(CLOSE, 1)
lc_df = pre_closed

# 计算DIF：CLOSE − IF(CLOSE > LC, MIN(LOW, LC), MAX(HIGH, LC))
min_values = np.minimum(low.values, lc_df.values)
max_values = np.maximum(high.values, lc_df.values)
dif_array = closed.values - np.where(closed.values > lc_df.values, min_values, max_values)

# 计算ACD：SUM(IF(CLOSE = LC, 0, DIF), 0)
acd_array = np.where(closed.values != lc_df.values, dif_array, 0)
acd_df = pd.DataFrame(acd_array, index=closed.index, columns=closed.columns).rolling(window=6).sum()

# 打印包含ACD的数据框
print(acd_df)

         000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0              NaN        NaN        NaN        NaN        NaN        NaN   
1              NaN        NaN        NaN        NaN        NaN        NaN   
2              NaN        NaN        NaN        NaN        NaN        NaN   
3              NaN        NaN        NaN        NaN        NaN        NaN   
4              NaN        NaN        NaN        NaN        NaN        NaN   
...            ...        ...        ...        ...        ...        ...   
2300 -2.100000e-01       0.18       0.17       0.15       0.35       1.17   
2301  8.881784e-16       0.17      -0.12       0.17       0.22       1.00   
2302  9.000000e-02       0.21       0.26       0.38       0.09       0.85   
2303 -1.100000e-01       0.16       0.35       0.35       0.11       3.51   
2304 -4.400000e-01      -0.22      -0.20       0.09      -0.28       2.95   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH

In [16]:
#计算ACD以20个交易日的数据

acd_df20 = pd.DataFrame(acd_array, index=closed.index, columns=closed.columns).rolling(window=20).sum()


以上是计算ACD 6日和20日的方法。

In [17]:
#计算ADTM动态买卖气指标，20个交易日

import pandas as pd
import numpy as np

# 计算DTM
dtm = np.maximum(high.values - open.values, open.values - open.shift(1, axis=0).fillna(0).values)

# 计算DBM
dbm = np.maximum(open.values - low.values, open.values - open.shift(1, axis=0).fillna(0).values)

# 计算STM和SBM（过去20天的和）
p = 20  # 过去20天的和
stm = pd.DataFrame(dtm).rolling(window=p, min_periods=1).sum().values
sbm = pd.DataFrame(dbm).rolling(window=p, min_periods=1).sum().values

# 计算ADTM
adtm = np.where(stm > sbm, (stm - sbm) / stm, np.where(stm == sbm, 0, (stm - sbm) / sbm))

# 将ADTM数据加入到数据框中
adtm_df = pd.DataFrame(adtm, index=open.index, columns=open.columns)

# 查看结果数据框
print(adtm_df)


      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
1     -0.054688  -0.003464   0.055676  -0.007720  -0.026609   0.065667   
2     -0.032203  -0.024000   0.055007  -0.015197   0.000851   0.077562   
3      0.049593  -0.060284   0.056708  -0.022444   0.019917   0.055896   
4      0.089177  -0.079345   0.070998  -0.029471   0.069020   0.141862   
...         ...        ...        ...        ...        ...        ...   
2300   0.027888   0.120879   0.056017   0.196078   0.115385   0.238189   
2301  -0.036290  -0.201878  -0.170923   0.082873   0.024896   0.141053   
2302  -0.052419  -0.202899  -0.154348   0.093567   0.054393   0.137856   
2303  -0.111538  -0.238318  -0.184549   0.172414   0.055794  -0.303544   
2304  -0.060976  -0.175355  -0.117241   0.216667   0.229839  -0.241491   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      0.000000   0.000000   0.0

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16068\960478978.py:18: RuntimeWarning: divide by zero encountered in divide
  adtm = np.where(stm > sbm, (stm - sbm) / stm, np.where(stm == sbm, 0, (stm - sbm) / sbm))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_16068\960478978.py:18: RuntimeWarning: invalid value encountered in divide
  adtm = np.where(stm > sbm, (stm - sbm) / stm, np.where(stm == sbm, 0, (stm - sbm) / sbm))


In [18]:
#AR\BR指标，20个交易日
import pandas as pd
import numpy as np

## 假设M为过去20天
M = 20

# 计算AR指标的分子（numerator）和分母（denominator）
ar_numerator = high - open
ar_denominator = open - low

# 处理NaN值，将NaN替换为0
ar_numerator = ar_numerator.where(~ar_numerator.isna(), 0)
ar_denominator = ar_denominator.where(~ar_denominator.isna(), 0)

# 计算AR指标
ar = (ar_numerator.rolling(window=M, min_periods=1).sum() / ar_denominator.rolling(window=M, min_periods=1).sum()) * 100

# 计算BR指标的分子（numerator）和分母（denominator）
br_numerator = np.maximum(0, high - pre_closed)
br_denominator = np.maximum(0, pre_closed - low)

# 处理NaN值，将NaN替换为0
br_numerator = br_numerator.where(~br_numerator.isna(), 0)
br_denominator = br_denominator.where(~br_denominator.isna(), 0)

# 计算BR指标
br = (br_numerator.rolling(window=M, min_periods=1).sum() / br_denominator.rolling(window=M, min_periods=1).sum()) * 100


In [19]:
#ARC指标，20个交易日
import pandas as pd
import numpy as np
# 假设M为20
M = 20

# 计算RC指标
rc = closed / closed.shift(M)

# 计算ARC指标
arc = rc.shift(1).rolling(window=M, min_periods=1).mean()


In [20]:
import pandas as pd

# 假设计算期天数为25
days = 20

# 计算Aroon上升数
aroon_up = ((days - high.rolling(window=days, min_periods=1).apply(lambda x: x.argmax(), raw=True)) / days) * 100

# 计算Aroon下降数
aroon_down = ((days - low.rolling(window=days, min_periods=1).apply(lambda x: x.argmin(), raw=True)) / days) * 100

# 计算Aroon指标
aroon = aroon_up - aroon_down

In [21]:
import pandas as pd
import numpy as np

# 计算LC（REF(C, 1)）
lc = closed.shift(1)

# 计算AA, BB, CC, DD
aa = abs(high - closed.shift(1))
bb = abs(low - closed.shift(1))
cc = abs(high - lc)
dd = abs(lc - open.shift(1))

# 计算R
r = np.where((aa > bb) & (aa > cc), (aa + bb / 2 + dd / 4),
             np.where((bb > cc) & (bb > aa), (bb + aa / 2 + dd / 4),
                      (cc + dd / 4)))

# 计算X
x = (closed - lc + (closed - open) / 2 + lc - open.shift(1))

# 计算SI
si = 16 * x / r * np.maximum(aa, bb)

# 计算ASI
asi = si.cumsum()


In [22]:
#ATR指标，以12天和20天为例

import pandas as pd
import numpy as np
# 计算ATR1
atr1 = np.maximum(np.maximum(high - low, np.abs(pre_closed - high)), np.abs(pre_closed - low))

# 假设M为移动平均的窗口大小，例如20
M1 = 12
M2 = 20

# 计算ATR指标
atr12 = atr1.rolling(window=M1, min_periods=1).mean()
atr20 = atr1.rolling(window=M2, min_periods=1).mean()


In [23]:
#BBI指标
import pandas as pd

# 假设M1、M2、M3、M4分别为3、6、12、24
M1, M2, M3, M4 = 3, 6, 12, 24

# 计算各条移动平均线
ma1 = closed.rolling(window=M1, min_periods=1).mean()
ma2 = closed.rolling(window=M2, min_periods=1).mean()
ma3 = closed.rolling(window=M3, min_periods=1).mean()
ma4 = closed.rolling(window=M4, min_periods=1).mean()

# 计算BBI指标
bbi = (ma1 + ma2 + ma3 + ma4) / 4


In [24]:
#BIAS指标，以6，12，24个交易日
import pandas as pd

# 假设P1为移动平均的时期，例如20
P1,P2,P3 = 6,12,24

# 计算移动平均线
ma6 = closed.rolling(window=P1, min_periods=1).mean()
ma12 = closed.rolling(window=P2, min_periods=1).mean()
ma24 = closed.rolling(window=P3, min_periods=1).mean()

# 计算乖离率
bias6 = ((closed - ma6) / ma6) * 100
bias12 = ((closed - ma12) / ma12) * 100
bias24 = ((closed - ma24) / ma24) * 100

In [25]:
#CCI指标，以20个交易日为例
import pandas as pd

# 假设M为移动平均的时期，例如20
M = 20

# 计算TYPP
typp = (high + low + closed) / 3

# 定义计算平均绝对偏差的函数
def calculate_avedev(x):
    return np.mean(np.abs(x - np.mean(x)))

# 计算TYPP的M日移动平均线
typp_ma = typp.rolling(window=M, min_periods=1).mean()

# 计算TYPP的M日平均绝对偏差
typp_avedev = typp.rolling(window=M, min_periods=1).apply(calculate_avedev, raw=True)

# 计算CCI指标
cci = (typp - typp_ma) / (0.015 * typp_avedev)


In [26]:
#Chaikin Volatility

import pandas as pd

# 假设high和low分别是高价和低价的数据框，window为计算EMA的窗口期，例如10
window = 10

# 计算10日HIGH-LOW的EMA
high_low_diff = high - low
ema_diff = high_low_diff.ewm(span=window, min_periods=0, adjust=False, ignore_na=False).mean()

# 计算10日前HIGH-LOW的EMA
ema_diff_shifted = high_low_diff.shift(window).ewm(span=window, min_periods=0, adjust=False, ignore_na=False).mean()

# 计算Chaikin Volatility
chaikin_volatility = ((ema_diff - ema_diff_shifted) / ema_diff_shifted) * 100

print(chaikin_volatility)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
2300  -2.900952 -20.517907 -32.034527  -7.147849 -19.102945  -1.507978   
2301  -1.566770 -23.409578 -31.954927 -15.280547 -18.818229  -3.469681   
2302  -2.332044 -26.361910 -33.112394 -15.958276 -16.130270  -2.910183   
2303  -1.899604 -30.981043 -38.772503 -14.631721 -20.511115 -10.691291   
2304  -4.742701  -8.794021 -30.219768 -16.945029 -11.390349  -9.127020   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

In [27]:
#钱德动量摆动指标,10个交易日

import pandas as pd

# 假设close为收盘价的数据框，N为计算SMA的窗口期，例如10
N = 10

# 计算CN1和CN2
CN1 = closed - pre_closed
CN1[CN1 < 0] = 0

CN2 = closed - pre_closed
CN2[CN2 > 0] = abs(CN2)
CN2[CN2 <= 0] = 0

# 计算SU和SD（过去N个交易日的SMA）
SU = CN1.rolling(window=N, min_periods=1).mean()
SD = CN2.rolling(window=N, min_periods=1).mean()

# 计算Chaikin摆动指标（CMO）
CMO = ((SU - SD) / (SU + SD)) * 100

print(CMO)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        0.0        NaN        NaN        NaN   
1           NaN        NaN        0.0        NaN        NaN        0.0   
2           NaN        0.0        0.0        NaN        0.0        0.0   
3           0.0        0.0        0.0        0.0        0.0        0.0   
4           0.0        0.0        0.0        0.0        0.0        0.0   
...         ...        ...        ...        ...        ...        ...   
2300        0.0        0.0        0.0        0.0        0.0        0.0   
2301        0.0        0.0        0.0        0.0        0.0        0.0   
2302        0.0        0.0        0.0        0.0        0.0        0.0   
2303        0.0        0.0        0.0        0.0        0.0        0.0   
2304        0.0        0.0        0.0        0.0        0.0        0.0   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           0.0        0.0      

In [28]:
#Coppock Curve估波指标

import pandas as pd

# 假设close为收盘价的数据框，N1、N2和N3分别为计算RC和WMA的窗口期，例如11、14、10
N1, N2, N3 = 11, 14, 10

# 计算RC（价格变化率）
RC1 = ((closed - closed.shift(N1)) / closed.shift(N1)) * 100
RC2 = ((closed - closed.shift(N2)) / closed.shift(N2)) * 100

# 计算RC的加权平均值
RC_combined = RC1 + RC2

# 计算WMA（加权移动平均）
def weighted_moving_average(data, window):
    weights = list(range(1, window + 1))
    wma = data.rolling(window=window, min_periods=1).apply(lambda x: np.dot(x, weights) / sum(weights), raw=True)
    return wma

# 计算估波指标（Coppock Curve）
coppock_curve = weighted_moving_average(RC_combined, N3)

# 查看结果数据框
print(coppock_curve)  # 去掉NaN值的部分


      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
2300   0.213322   1.082313  -8.989520   3.801476   7.497128   7.257136   
2301   3.097196   2.925814  -7.861772   5.213577   9.073565  10.015094   
2302   4.109328  -0.149373  -7.365107   6.076610   9.389281  11.764455   
2303   5.731371  -2.141517  -6.124828   7.583503   9.913095  13.606040   
2304   5.864124  -4.372418  -5.454776   7.944354   9.587288   6.830870   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

In [29]:
#)CR能量指标

import pandas as pd

# 假设high、low、close分别为最高价、最低价和收盘价的数据框，N为计算CR指标的周期，例如20
N = 20

# 计算中间价（MID）
mid = (high + low + closed) / 3

# 计算CR指标的分子和分母
numerator = mid - mid.shift(1)
denominator_high = high - mid.shift(1)
denominator_low = mid.shift(1) - low

# 计算CR指标
cr_numerator = numerator.rolling(window=N, min_periods=1).sum()
cr_denominator_high = denominator_high.rolling(window=N, min_periods=1).sum()
cr_denominator_low = denominator_low.rolling(window=N, min_periods=1).sum()

cr = (cr_numerator / cr_denominator_high) * 100  # 当denominator_high为0时，cr_denominator_low不为0
cr[cr_denominator_high == 0] = (cr_numerator[cr_denominator_high == 0] / cr_denominator_low[cr_denominator_high == 0]) * 100

# 查看结果数据框
print(cr)  


      000009.SZ   000012.SZ  000021.SZ   000027.SZ  000031.SZ     000039.SZ  \
0           NaN         NaN        NaN         NaN        NaN           NaN   
1    -50.000000  -11.382114  44.808743    0.000000 -19.298246     51.315789   
2    -20.754717  -20.161290  20.717131    0.000000   3.797468     38.511327   
3     10.192837 -185.483871  14.516129  634.198715  12.173913     15.901060   
4     26.929982 -125.603865  19.750520   -0.000000  20.145631     30.807249   
...         ...         ...        ...         ...        ...           ...   
2300   7.084020    3.651685  -2.065217   19.502075   6.773619     23.180077   
2301   2.659574  -45.049505 -16.974170   12.530120   3.307393     20.184426   
2302  -0.561798  -77.018634 -22.314050   13.519814   6.826568     20.396040   
2303  -3.518519  -80.503145 -23.347398   16.816143   8.921933   -136.444444   
2304  -1.957295  -56.000000 -21.852388   18.965517  15.806988 -13350.000000   

       000050.SZ   000060.SZ  000066.SZ   000089.SZ

In [30]:
#DBCD异同离差乖离率

import pandas as pd

# 假设close为收盘价的数据框，P为计算BIAS的窗口期，N为计算DIF的周期，W为SMA的窗口期，例如20、5、5
P, N, W = 20, 5, 5

# 计算BIAS（乖离率）
bias = (closed - closed.rolling(window=P).mean()) / closed.rolling(window=P).mean()

# 计算DIF（不同日的乖离率之间的离差）
dif = bias - bias.shift(N)

# 计算DBCD指标（对离差进行指数移动平滑处理）
dbcd = dif.ewm(span=W, min_periods=1).mean()

# 查看结果数据框
print(dbcd)  


      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
2300   0.022678   0.027375   0.011594   0.031028   0.045441   0.047698   
2301   0.038695   0.030750   0.018475   0.038021   0.038789   0.052294   
2302   0.034077   0.000136   0.019654   0.035150   0.027141   0.048322   
2303   0.023383  -0.029180   0.013026   0.028247   0.010646   0.038303   
2304   0.007412  -0.054156  -0.000691   0.016505  -0.004885  -0.045120   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

In [31]:
#PDI,MDI,ADX,ADXR    DMI动向指标
import pandas as pd

high_change = high - high.shift(1)
low_change = low - low.shift(1)
hl= high - low
abs_high_close_change = (high - pre_closed).abs()
abs_low_close_change = (low - pre_closed).abs()

# 计算最大值
# 在每个位置上找到最大值
max_values = np.maximum( hl, abs_high_close_change, abs_low_close_change)

P = 14  # 假设P为14
TRI = max_values.rolling(window=P, min_periods=1).sum()

# 计算PDI
conditional_hd = high_change.where(((high_change > 0) & (high_change > low_change)), 0)
# 计算DMP，使用rolling窗口大小为P计算滚动和
DMP = conditional_hd.rolling(window=P, min_periods=1).sum()

PDI = DMP *100 / TRI

# 计算MDI
conditional_ld = high_change.where(((low_change > 0) & (high_change < low_change)), 0)
# 计算DMP，使用rolling窗口大小为P计算滚动和
DMM = conditional_ld.rolling(window=P, min_periods=1).sum()

MDI = DMM *100 / TRI

M=20
#计算ADX,ADXR

mp = MDI-PDI;pm = MDI+PDI;adxx = mp.abs()/pm

ADX = adxx.rolling(window=M, min_periods=1).mean()
ADXR= (ADX+ADX.shift(M))/2

In [32]:
#Elder 透视指标 10个交易日

def calculate_ema(close, n):
    ema = close.ewm(span=n, min_periods=n).mean()
    return ema

long=high-calculate_ema(closed,10)
short=low-calculate_ema(closed,10)
Elder=(long-short)/closed
print(Elder)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
2300   0.069204   0.027719   0.041420   0.017685   0.030395   0.023014   
2301   0.026891   0.012876   0.031325   0.012862   0.015385   0.010417   
2302   0.022375   0.011820   0.019231   0.012862   0.021638   0.016393   
2303   0.032759   0.023753   0.018116   0.017572   0.015432   0.014147   
2304   0.017513   0.050481   0.030750   0.011290   0.026480   0.018727   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

In [33]:
#Hurst指数,10个交易日
M=10
m=closed.rolling(window=M, min_periods=1).mean()
YI=closed - m
def zn(YI,t):
    zn=YI.rolling(window=t, min_periods=1).sum()
    return zn

# 将这些数据框放入一个列表
dataframes_list = [zn(YI, 1), zn(YI, 2), zn(YI, 3), zn(YI, 4), zn(YI, 5), zn(YI, 6), zn(YI, 7), zn(YI, 8), zn(YI, 9), zn(YI, 10)]

# 计算最大值
max = np.maximum.reduce([df.values for df in dataframes_list])
min = np.minimum.reduce([df.values for df in dataframes_list])
RN=max-min
SN=closed.rolling(window=M, min_periods=1).std()
HURST=RN/SN

print(HURST)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1      0.000000   0.000000   0.000000        NaN   0.000000   0.000000   
2      0.850439   0.558014   0.974355        NaN   0.902597   0.427121   
3      1.673514   0.631308   1.160069   0.000000   1.608522   1.801078   
4      0.817301   1.094427   2.247791   1.677051   1.530885   2.122808   
...         ...        ...        ...        ...        ...        ...   
2300   4.912862   4.823089   7.588111   4.009145   9.856621   5.918510   
2301   4.065283   4.372426   6.556055   4.219878   9.769413   6.426882   
2302   5.414317   2.889320   8.001671   5.169707  10.441751   7.076917   
2303   5.821174   2.317268   8.366317   5.779763   9.478683   7.751570   
2304   6.601965   3.014028   5.026589   7.324869  10.900280   4.440995   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

In [34]:
#KDJ指标
#K与D值永远介于0到100之间。D大于70时，行情呈现超买现象。D小于30时，行情呈现超卖现象。
import pandas as pd

# 定义周期N
N = 10

# 计算RSV值
high_n = high.rolling(window=N, min_periods=1).max()
low_n = low.rolling(window=N, min_periods=1).min()
rsv = (closed - low_n) / (high_n - low_n) * 100

# 计算K值（使用5日的简单移动平均）
P1 = 5
k_values = rsv.rolling(window=P1, min_periods=1).mean()

# 计算D值（使用3日的简单移动平均）
P2 = 3
d_values = k_values.rolling(window=P2, min_periods=1).mean()

# 计算J值
j_values = 3 * k_values - 2 * d_values


# 输出KDJ指标数据框
print(j_values)


      000009.SZ  000012.SZ  000021.SZ    000027.SZ   000031.SZ   000039.SZ  \
0     77.551020  85.000000  20.000000    81.539087  109.302326   50.000000   
1     32.758621  23.076923  49.450549    81.539087   48.979592   45.794393   
2     23.251753  68.682336  48.302808    81.539087   31.183006   62.305296   
3     41.463373  60.773301  50.189255  1396.274860   58.933757   55.646417   
4     61.987988  39.331909  49.284493   757.688914   58.547853   43.700085   
...         ...        ...        ...          ...         ...         ...   
2300  69.538646  81.441793  57.496239    83.008012   74.482620   83.029225   
2301  83.621028  91.429845  58.451178   101.315939   67.816607   96.829191   
2302  74.599263  61.501274  57.329019    99.371268   61.712092   99.033129   
2303  59.061422  33.658226  46.032208    98.090040   50.938889  100.506288   
2304  41.203655  12.183227  21.303600    86.100124   34.061836   57.230278   

      000050.SZ    000060.SZ  000066.SZ  000089.SZ  ...  688536

In [35]:
#Klinger 成交量摆动指标，若今日均价高于昨日均价则成交量记为正，反之记为负。然后计算55期和34期内经上述处理过的成交量的指数平均数，将两平均数相减得到差值。然后计算该差值的6日指数平均值

average = (closed+high+low)/3

aver55 = average.ewm(span=55, min_periods=1, adjust=False).mean()
aver34 = average.ewm(span=34, min_periods=1, adjust=False).mean()

aver = aver55-aver34
Klinger = aver.ewm(span=6, min_periods=1, adjust=False).mean()

print(Klinger)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
1      0.001163   0.000286  -0.001673   0.000000   0.000224  -0.002388   
2      0.002560   0.001484  -0.003775   0.000000   0.000242  -0.006300   
3      0.002639   0.006914  -0.005951  -0.009473  -0.000326  -0.009582   
4     -0.000447   0.015552  -0.009133  -0.015359  -0.002379  -0.015256   
...         ...        ...        ...        ...        ...        ...   
2300   0.041413   0.090444  -0.065231   0.013114  -0.027318   0.149731   
2301   0.040090   0.090528  -0.060451   0.010646  -0.027299   0.138617   
2302   0.038650   0.092380  -0.055759   0.007765  -0.027267   0.126546   
2303   0.037335   0.095543  -0.050545   0.004750  -0.026825   0.125540   
2304   0.036034   0.098923  -0.045780   0.001525  -0.026541   0.136367   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      0.000000   0.000000   0.0

In [36]:
#均线价格比（Moving Average Ratio）,选择10日、20日来计算，指数平均
EMA10=closed.ewm(span=10, min_periods=1, adjust=False).mean()
EMA20=closed.ewm(span=20, min_periods=1, adjust=False).mean()

MAR10=closed/EMA10
MAR20=closed/EMA20

print(MAR10)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      1.000000   1.000000   1.000000   1.000000   1.000000   1.000000   
1      0.978766   0.986047   1.018659   1.000000   0.982780   1.011223   
2      0.981795   1.013490   1.009648   1.000000   0.987373   1.024017   
3      1.000333   0.966954   1.018370   1.409131   1.000180   1.006617   
4      1.039513   0.964186   1.013751   0.924362   1.006893   1.006643   
...         ...        ...        ...        ...        ...        ...   
2300   1.012269   1.019983   1.008827   1.027563   1.022334   1.046125   
2301   1.034136   1.010985   0.992557   1.022439   1.008089   1.035549   
2302   1.008007   0.931638   0.995864   1.018285   1.002810   1.027654   
2303   1.005126   0.939665   0.992687   1.020227   1.003564   1.023133   
2304   0.991417   0.940734   0.979207   1.008532   0.995308   0.842168   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      1.000000   1.000000   1.0

In [37]:
#MACD指数平滑异动平均线，取10日和20日。后做15日EMA。

DIFF = EMA10 - EMA20

DEA = DIFF.ewm(span=15, min_periods=1, adjust=False).mean()
MACD = 2 * (DIFF-DEA)

print(MACD)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
1     -0.042424  -0.050000   0.045455   0.000000  -0.034848   0.027273   
2     -0.069307   0.009497   0.058997   0.000000  -0.052656   0.079944   
3     -0.052874  -0.109906   0.090492   0.703275  -0.040281   0.078203   
4      0.039432  -0.211990   0.103130   0.420519  -0.016434   0.075981   
...         ...        ...        ...        ...        ...        ...   
2300   0.106661   0.098974  -0.020907   0.098426   0.157940   0.426142   
2301   0.131555   0.097895  -0.022776   0.107820   0.144882   0.444250   
2302   0.121872   0.029020  -0.018858   0.109377   0.126518   0.434732   
2303   0.109686  -0.018431  -0.020429   0.111607   0.111379   0.411377   
2304   0.084101  -0.054252  -0.042103   0.098775   0.088202  -0.038901   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      0.000000   0.000000   0.0

In [38]:
#Mass Index梅斯线,该指标最主要的作用，在于寻找飙涨股或者极度弱势股的重要趋势反转点。
#𝑀𝑎𝑠𝑠 𝐼𝑛𝑑𝑒𝑥 = EMA(HIGH − LOW, 9)/EMA((EMA(HIGH − LOW, 9), 9)

hl= high - low
EMAHL9=hl.ewm(span=9, min_periods=1, adjust=False).mean()
EMAEMAHL9 =EMAHL9.ewm(span=9, min_periods=1, adjust=False).mean()
MASS=EMAHL9/EMAEMAHL9
print(MASS)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      1.000000   1.000000   1.000000   1.000000   1.000000   1.000000   
1      0.963756   1.044825   0.956873   1.000000   1.000000   0.976639   
2      1.081975   1.280039   0.927552   1.000000   1.025878   0.967552   
3      1.136172   1.136977   0.905618   1.000000   0.982176   0.916983   
4      1.099207   1.109314   0.935887   1.000000   1.107819   1.005889   
...         ...        ...        ...        ...        ...        ...   
2300   1.151667   0.941293   0.904436   0.965806   0.991506   1.031796   
2301   1.080931   0.889311   0.917938   0.920517   0.930892   0.945273   
2302   1.007636   0.839962   0.871896   0.892323   0.933546   0.940558   
2303   1.013842   0.894965   0.837165   0.919181   0.895651   0.919986   
2304   0.936508   1.114399   0.880665   0.882085   0.948451   0.915452   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      1.000000   1.000000   1.0

AD指标、Chaikin Oscillator佳庆指标、Ease of Movement简易波动指标缺少成交量无法计算。DDI指标无法计算。DBTP指标,MA均价线性回归系数没看懂。JDQS阶段强势指标缺少关键指标无法计算。


后三十个指标。

In [39]:
# 32 MICD


#计算MTM（动力指标）
MTM = closed - closed.shift(1)
# 10日移动平均线
MTMMA = MTM.rolling(window = 10).mean()

# Step 3: 计算DIF（离差值）
N1 = 12  # N1日移动平均
N2 = 26  # N2日移动平均
DIF = MTMMA .shift(1).rolling(window = N1).mean() - MTMMA.shift(1).rolling(window= N2).mean()

#计算MICD（动力指数离差平均线）
MICD = DIF.rolling(window = 10).mean()

In [40]:
# 34 MTM

# m 基于关注价格动向的期限：短期（10），中期（50），长期（200）
m = 10
MTM = closed - closed.shift(m)
MTMMA = MTM.rolling(window = m).mean()

In [41]:
# 36 PSY

M = 12 #自定义周期
condition = closed > closed.shift(1)
# 使用滚动窗口来计算条件为 True 的数量，即收盘价大于前一周期的次数
count_true = condition.rolling(window = M).sum()
PSY = count_true / M * 100 

In [42]:
# 37 PVI正成交量指标

condition = closed > closed.shift(1)
true = (closed - closed.shift(1))/closed.shift(1)
PVI = np.where(condition, true, closed.shift(1))
PVI = pd.DataFrame(PVI, index=closed.index, columns=closed.columns)

In [43]:
# 39 RCCD异同离差变化率指数

m = 10
RC = closed/closed.shift(m)
ARC1 = closed.shift(1).rolling(window = m).mean()

p1 = 6
p2 = 12
DIF = ARC1.shift(1).rolling(window = p1).mean()-ARC1.shift(1).rolling(window = p2).mean()
RCCD = DIF.shift(1).rolling(window = m).mean()

In [44]:
# 40 RC变化率指数
m = 12
RC = closed/closed.shift(m)

In [45]:
# 41 ROC变动速率 是以今天之价格比较其N天前之价格的变化率。采用12天及25天周期可达到相当的效果。

N = 12
ROC = (closed - closed.shift(N))/closed.shift(N)
print(ROC)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
2300   0.037702   0.042222  -0.028736   0.052453   0.073409   0.094232   
2301   0.110075   0.073733   0.023428   0.070568   0.067323   0.108911   
2302   0.054446  -0.029817   0.020859   0.043624   0.058920   0.094617   
2303   0.012216  -0.108051  -0.020118   0.031301   0.017268   0.063341   
2304   0.000000  -0.099567  -0.055749   0.033333   0.022293  -0.151033   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

In [46]:
# 42 RSI 相对强弱指标

N1 = 14

PriceChange = closed - pre_closed
AbsoluteChange = abs(PriceChange)
# 将负数转换为0
PriceChange[PriceChange < 0] = 0

# 计算 RSI
RSI = (PriceChange.rolling(N1).mean() / AbsoluteChange.rolling(N1).mean()) * 100

print(RSI)


      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
2300  55.303030  55.769231  40.000000  67.346939  72.000000  79.512195   
2301  62.500000  56.862745  41.818182  70.212766  67.289720  77.941176   
2302  57.692308  59.615385  41.284404  69.565217  65.454545  77.073171   
2303  66.176471  68.888889  58.181818  79.069767  68.224299  85.026738   
2304  57.692308  64.516129  52.272727  68.831169  63.963964  81.460674   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

In [47]:
# 43 RVI相对波动率指数

N = 14
condition1 = closed > closed.shift(1)
condition2 = closed <= closed.shift(1)

up = pd.DataFrame(np.where(condition1, closed.rolling(window=N).std(), 0), columns=closed.columns, index=closed.index)
down = pd.DataFrame(np.where(condition2, closed.rolling(window=N).std(), 0), columns=closed.columns, index=closed.index)

AUP = up.rolling(N).mean()
ADOWN = down.rolling(N).mean()
RVI = AUP/(AUP+ADOWN)*100


In [48]:
# 44 SRMI修正动量指标

M = 7  # 可根据需要调整M的值

condition = closed < closed.shift(M)
true_value = (closed - closed.shift(M))/closed.shift(M)

condition1 = closed == closed.shift(M)
false_value = np.where(condition1,0,(closed - closed.shift(M))/closed)
false_value_df = pd.DataFrame(false_value, index=closed.index, columns=closed.columns)
# 计算变化率指标
SRMI = np.where(condition,true_value,false_value_df)
SRMI_df = pd.DataFrame(SRMI, index=closed.index, columns=closed.columns)

In [49]:
# 45 StochRSI随机强弱指数

#StochRSI=（当日RSI-周期内最小RSI）/（周期内最大RSI-周期内最小RSI）

N = 14  # 可根据需要调整周期N的值
stoch_rsi_min = RSI.rolling(N).min()
stoch_rsi_max = RSI.rolling(N).max()
stoch_rsi = (RSI - stoch_rsi_min) / (stoch_rsi_max - stoch_rsi_min)

In [50]:
# 46 TEMA三重指数移动平均线

N = 14  # 可根据需要调整N的值

# 计算EMA1
ema1 = closed.ewm(span=N, adjust=False).mean()
# 计算EMA2
ema2 = ema1.ewm(span=N, adjust=False).mean()
# 计算EMA3
ema3 = ema2.ewm(span=N, adjust=False).mean()

# 计算TEMA
tema = 3 * ema1 - 3 * ema1 + ema3

print(tema)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0     10.840000  19.410000  13.100000   8.438387  10.970000  13.090000   
1     10.839336  19.409218  13.100711   8.438387  10.969455  13.090427   
2     10.837587  19.408583  13.102347   8.438387  10.968085  13.092105   
3     10.835008  19.406228  13.105400   8.449390  10.966083  13.095009   
4     10.833046  19.400555  13.110068   8.466993  10.963824  13.099103   
...         ...        ...        ...        ...        ...        ...   
2300   5.792510   4.676317   8.495765   6.037699   6.469776  12.824104   
2301   5.782399   4.666526   8.488797   6.034196   6.459442  12.807789   
2302   5.774180   4.657156   8.481519   6.032407   6.451192  12.798239   
2303   5.767653   4.647454   8.473915   6.032360   6.444767  12.795054   
2304   5.762408   4.636852   8.465652   6.033847   6.439784  12.791154   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      6.750000  28.200000  16.2

In [51]:
# 47 TRIX三重指数平滑平均线

N = 14
# 计算对数价格
log_closed = np.log(closed)
TRIX = log_closed.ewm(span=N, adjust=False).mean().ewm(span = N, adjust=False).mean().ewm(span=N, adjust=False).mean()

print(TRIX)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      2.383243   2.965788   2.572612   2.132791   2.395164   2.571849   
1      2.383181   2.965748   2.572666   2.132791   2.395114   2.571881   
2      2.383017   2.965714   2.572789   2.132791   2.394988   2.572008   
3      2.382777   2.965590   2.573020   2.133830   2.394804   2.572227   
4      2.382590   2.965290   2.573372   2.135492   2.394596   2.572536   
...         ...        ...        ...        ...        ...        ...   
2300   1.755160   1.540940   2.138287   1.797704   1.865878   2.549699   
2301   1.753507   1.538950   2.137535   1.797137   1.864343   2.548537   
2302   1.752174   1.537033   2.136743   1.796850   1.863130   2.547890   
2303   1.751129   1.535020   2.135910   1.796849   1.862199   2.547727   
2304   1.750300   1.532783   2.134994   1.797100   1.861489   2.547462   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      1.909543   3.339322   2.7

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [52]:
# 48 TRI 区间指数

T = 10 #区间

# 计算X1
X1 = np.where(closed > closed.shift(1), T / (closed - closed.shift(1)), T)
# 计算X2
X2 = X1 - X1.min()

# 计算TRI
N = 14  # EMA的周期
TRI = (X2 / (high - low) * 100).ewm(span=N, adjust=False).mean()

print(TRI)

         000009.SZ      000012.SZ     000021.SZ     000027.SZ      000031.SZ  \
0      2026.912414    1655.311805   1324.249444   4822.628996    2309.737402   
1      2105.144121    1606.584011   1954.112043   4822.628996    2309.737402   
2      1987.945824    1529.616902   1938.795149   4822.628996   15333.288970   
3      2587.507520    1590.517871   3186.837853   4314.684357   16087.299157   
4      2698.759852    1548.224391   2951.104631   4382.410309   15890.448426   
...            ...            ...           ...           ...            ...   
2300  41140.340450  104684.809779  33641.052979  98521.041178  136803.357751   
2301  40551.245076   92933.917548  29664.905957  87040.214159  119887.159494   
2302  36163.065818   83191.227429  67370.574399  77090.164075  104848.097355   
2303  32038.297100   73423.313215  59270.664108  97106.247753  224192.600484   
2304  29090.773597   64264.133093  51897.608671  86050.532972  195079.224014   

         000039.SZ    000050.SZ      00

In [53]:
# 49 Ulcer 指标
# 以及定义窗口大小 n
n = 14 

# 计算 n 日内的最高价
highest_high = closed.rolling(window=n, min_periods=1).max()

# 计算每日收盘价相对于 n 日内最高价的变动率 Ri
Ri = (closed - highest_high) / highest_high*100

# 计算 Ri 平方的均值
Ri_squared_mean = (Ri ** 2).rolling(window=n, min_periods=1).mean()

# 计算 Ulcer 指标
ulcer_index = Ri_squared_mean.pow(0.5)

print(ulcer_index)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
1      1.826475   1.202191   0.000000   0.000000   1.482539   0.000000   
2      2.147023   0.981585   0.387773   0.000000   1.639150   0.000000   
3      1.905155   2.776933   0.335821   0.000000   1.454958   0.777202   
4      1.704022   3.757288   0.307609  15.869972   1.301354   0.937447   
...         ...        ...        ...        ...        ...        ...   
2300   5.108566   4.404606   7.609621   2.750503   5.225604   2.982133   
2301   4.873328   4.133451   7.621664   2.331918   4.085651   2.556836   
2302   4.638133   4.833583   7.604719   2.125517   3.441393   2.327411   
2303   3.920042   5.224450   7.153671   1.603540   2.655969   1.814820   
2304   3.598021   5.779723   6.806676   1.444856   1.713007   5.707419   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0      0.000000   0.000000   0.0

In [54]:
# 50 UOS终极波动指标

N1 = 7
N2 = 14
N3 = 28

TH = (high-closed.shift(1)).rolling(N1).max()
TL = (closed.shift(1)-low).rolling(N1).min()

ACC1 = (closed - TL.rolling(N1).mean()) / (TH-TL).rolling(N1).sum()
ACC2 = (closed - TL.rolling(N2).mean()) / (TH-TL).rolling(N2).sum()
ACC3 = (closed - TL.rolling(N3).mean()) / (TH-TL).rolling(N3).sum()

# 计算 UOS
UOS = (ACC1 * N2 * N3 + ACC2 * N1 * N3 + ACC3 * N1 * N2) * 100 / (N1 * N2 + N1 * N3 + N2 * N3)

print(UOS)

       000009.SZ   000012.SZ   000021.SZ   000027.SZ   000031.SZ   000039.SZ  \
0            NaN         NaN         NaN         NaN         NaN         NaN   
1            NaN         NaN         NaN         NaN         NaN         NaN   
2            NaN         NaN         NaN         NaN         NaN         NaN   
3            NaN         NaN         NaN         NaN         NaN         NaN   
4            NaN         NaN         NaN         NaN         NaN         NaN   
...          ...         ...         ...         ...         ...         ...   
2300  133.907918  105.585904  138.666973  171.925755  151.149499  184.745768   
2301  142.031463  111.090117  145.790114  170.032357  148.496709  179.316394   
2302  142.158915  107.894813  155.598304  166.181856  146.634744  173.344055   
2303  145.559621  115.832903  165.856101  163.080404  145.898808  168.201236   
2304  146.476977  123.994529  176.240956  162.293719  144.205562  132.125055   

       000050.SZ   000060.SZ   000066.S

In [55]:
# (58) W%R威廉指标

N = 6
P1 = 12
# 计算 RSV
HHV_HIGH = high.rolling(window=N).max()
LLV_LOW = low.rolling(window=N).min()
RSV = (HHV_HIGH - closed) / (HHV_HIGH - LLV_LOW) * 100

# 计算 W%R
WR = RSV.rolling(window=P1).mean()

print(WR)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
2300  46.094661  50.149113  60.714174  48.852360  43.024273  37.312561   
2301  39.905597  43.819264  58.440266  42.030771  42.731233  30.312500   
2302  38.715628  44.186327  56.409887  40.479545  42.482315  26.129507   
2303  42.536550  50.754428  57.525158  40.931847  46.754200  25.961548   
2304  47.877862  56.105641  62.485476  43.947198  50.431055  32.712655   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

In [56]:
# (60) 价格绝对方差均值 MA(ABS(CLOSE-MA(CLOSE，M))，M)
M = 10
PAVA = abs(closed-closed.rolling(window = M).mean()).rolling(window = M).mean()

print(PAVA)

      000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
2300     0.0752     0.0724     0.1558     0.0931     0.1479     0.2831   
2301     0.0929     0.0627     0.1406     0.0995     0.1395     0.3232   
2302     0.0966     0.0882     0.1402     0.1100     0.1341     0.3555   
2303     0.0922     0.1126     0.1320     0.1192     0.1199     0.3801   
2304     0.0949     0.1404     0.1122     0.1248     0.1191     0.5961   

      000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0           NaN        NaN      

MFI、Money flow、OBV、PVT、VEMA、VMACD、VOSC、VROC、VRSI、VR、VSTD缺少成交量无法计算。价格线性回归系数未计算。

In [57]:
#之前的closed和pre_closed都已经标准化，现在需要最初的数据。

# 读取closed.xlsx和pre_closed.xlsx文件，并将它们保存为数据框
closed_file_path = os.path.join(current_directory, 'closed.xlsx')
pre_closed_file_path = os.path.join(current_directory, 'pre_closed.xlsx')

# 使用文件名作为数据框的名称
closed = pd.read_excel(closed_file_path)
pre_closed = pd.read_excel(pre_closed_file_path)

date=closed['Date']
closed = closed.drop(columns='Date')
pre_closed = pre_closed.drop(columns='Date')


closed = closed.fillna(0)
pre_closed = pre_closed.fillna(0)

print(date)

0      2010-01-04
1      2010-01-05
2      2010-01-06
3      2010-01-07
4      2010-01-08
          ...    
2300   2019-06-24
2301   2019-06-25
2302   2019-06-26
2303   2019-06-27
2304   2019-06-28
Name: Date, Length: 2305, dtype: datetime64[ns]


In [58]:
#计算收益率，和下一个交易日的收益率
import numpy as np

logreturn = np.log(closed / pre_closed)

logreturnnext = logreturn.iloc[1:]

data = [[0] * 500]
df = pd.DataFrame(data, columns=logreturnnext.columns)

# 使用pd.concat()函数将两个数据框按行连接
logreturnnext = pd.concat([logreturnnext, df])

#print(logreturnnext)  得到一个一样大小的数据框，并且对应位置储存的是下一个交易日的收益率。


In [59]:
#计算每个指标的IC值，IC均值，ICIR。

import pandas as pd
from scipy.stats import spearmanr

# 计算信息系数
def get_IC(return_df, factor_df):
    # 计算预测值与实际值的协方差
    covariance = ((return_df - return_df.mean()) * (factor_df - factor_df.mean())).mean()
    
    # 计算预测值和实际值的标准差
    std_predictions = return_df.std()
    std_actuals = factor_df.std()
    
    # 计算信息系数
    information_coefficient = covariance / (std_predictions * std_actuals)
    
    return information_coefficient

# 计算信息系数
information_coefficient = get_IC(logreturnnext, MASS)

# 打印前10只股票的信息系数
print(information_coefficient.head(10))


000009.SZ    0.035099
000012.SZ   -0.008155
000021.SZ    0.004855
000027.SZ    0.030517
000031.SZ    0.067660
000039.SZ    0.037203
000050.SZ    0.031707
000060.SZ    0.104691
000066.SZ    0.062392
000089.SZ    0.097710
dtype: float64


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [60]:
#现在确定每一期的数据观测值数量。

# 创建包含日期序列的数据框
df = pd.DataFrame({'Date': date})

# 按照每半年进行重采样，并统计每个重采样期间的交易日数量
half_yearly_trade_days = df.groupby(pd.Grouper(key='Date', freq='6M', closed='left')).size()

# 打印每半年的交易日数量
print(half_yearly_trade_days)

Date
2010-06-30    117
2010-12-31    124
2011-06-30    119
2011-12-31    126
2012-06-30    117
2012-12-31    125
2013-06-30    114
2013-12-31    124
2014-06-30    119
2014-12-31    126
2015-06-30    119
2015-12-31    125
2016-06-30    120
2016-12-31    125
2017-06-30    118
2017-12-31    126
2018-06-30    119
2018-12-31    124
2019-06-30    118
Freq: 6M, dtype: int64


In [61]:
from itertools import accumulate

new_list = list(accumulate(half_yearly_trade_days))

print(new_list)

[117, 241, 360, 486, 603, 728, 842, 966, 1085, 1211, 1330, 1455, 1575, 1700, 1818, 1944, 2063, 2187, 2305]


In [62]:
# 计算每一期的信息系数，一共19期
start_index = [0,117, 241, 360, 486, 603, 728, 842, 966, 1085, 1211, 1330, 1455, 1575, 1700, 1818, 1944, 2063, 2187]
end_index = [116,240,359,485, 602, 727, 841, 965, 1084, 1210, 1329, 1454, 1574, 1699, 1817, 1943, 2062, 2186, 2304]

def period_IC(return_df,factor_df):
    ic_results = []
    for i in range(0, len(start_index)):
        period_start = start_index[i]
        period_end = end_index[i]  
        # 按照每一期的时间范围切片数据
        return_period = return_df[period_start:period_end]
        factor_period = factor_df[period_start:period_end]
            
        # 计算信息系数
        ic_values = get_IC(return_period, factor_period)
           
        # 将信息系数存储到列表中
        ic_results.append(ic_values)

    # 创建包含信息系数的数据框
    ic_df = pd.DataFrame(ic_results)
    return(ic_df)

ic_df = period_IC(logreturnnext,MASS)

# 打印信息系数的数据框
print(ic_df)


    000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0    0.012584  -0.157611  -0.087486   0.069208  -0.236235  -0.117558   
1    0.092904   0.106996   0.180168   0.119037   0.200476   0.271703   
2    0.213680   0.303371  -0.242553   0.144755   0.201917   0.252607   
3   -0.133702  -0.176037   0.102120  -0.079857  -0.025571   0.063554   
4    0.012816   0.097149   0.066792   0.039559   0.301433   0.019127   
5   -0.023330  -0.094999   0.005147  -0.132311   0.129077  -0.125714   
6    0.129219   0.143746   0.201668  -0.132091   0.257049   0.065524   
7   -0.026829   0.048080   0.103788   0.222573   0.148855   0.315898   
8    0.078818  -0.108916   0.129710   0.103887   0.009353  -0.045456   
9    0.260169   0.112882   0.025428   0.235764   0.453195   0.288637   
10   0.081408   0.242046   0.029193   0.222095   0.156456  -0.020458   
11  -0.054647  -0.092166  -0.124466  -0.196729   0.059373   0.036045   
12  -0.123949  -0.028025  -0.012131  -0.302898  -0.083213  -0.12

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric(

In [63]:
# 计算每一行的均值和标准差，然后计算均值除以标准差的商
row_means = ic_df.mean(axis=1)  # 计算每一行的均值
row_std = ic_df.std(axis=1)      # 计算每一行的标准差

# 计算均值除以标准差的商
result_df = pd.DataFrame({
    'MASA_IC_Mean': row_means,
    'MASS_IC_Std': row_std,
    'MASS_IC_ICIR': row_means / row_std
})

# 输出结果数据框
print(result_df)

    MASA_IC_Mean  MASS_IC_Std  MASS_IC_ICIR
0       0.013871     0.136552      0.101582
1       0.122335     0.130022      0.940874
2       0.053319     0.141992      0.375507
3       0.027311     0.146061      0.186983
4       0.070068     0.138717      0.505115
5       0.079569     0.130218      0.611047
6       0.047276     0.151988      0.311051
7       0.137856     0.115902      1.189420
8       0.067270     0.141628      0.474975
9       0.177907     0.127519      1.395139
10      0.106872     0.119195      0.896610
11     -0.075249     0.123269     -0.610448
12     -0.081099     0.127662     -0.635268
13      0.136023     0.146470      0.928672
14      0.033007     0.159425      0.207040
15      0.096576     0.131089      0.736717
16     -0.064192     0.154641     -0.415102
17     -0.039329     0.149057     -0.263851
18      0.195358     0.103634      1.885079


以上的IC、IC均值、ICIR是用翌日收益率和MASS指标计算的。如果想知道所有指标，就需要重复使用。最后统一一下。

In [64]:
factor_list = [acd_df,acd_df20,adtm_df,ar,br,arc,aroon,asi,atr12,atr20,bbi,bias6,bias12,bias24,cci,chaikin_volatility,CMO,coppock_curve,cr,dbcd,ADX,ADXR,Elder,HURST,j_values,Klinger,MAR10,MAR20,MACD,MASS,MICD,MTMMA,PSY,PVI,RCCD,RC,ROC,RVI,SRMI_df,tema,TRIX,TRI,UOS,WR,PAVA,RSI,stoch_rsi,ulcer_index]#所有因子数据框
IC_list=[]
for factor_df in factor_list:
    ic_df=period_IC(logreturnnext,factor_df)
    row_means = ic_df.mean(axis=1)
    row_std = ic_df.std(axis=1)
    result_df = pd.DataFrame({
    'IC_Mean': row_means,
    'IC_Std': row_std,
    'IC_ICIR': row_means / row_std})
    IC_list.append(result_df)

    

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric(

In [65]:
print(len(factor_list))

48


In [66]:
print(len(IC_list))

48


In [67]:
import pandas as pd

# 假设你的数据框列表叫做 dataframe_list
dataframe_list = IC_list


In [68]:
print(len(dataframe_list))

48


In [69]:
import pandas as pd

# Assuming you have a list of dataframes called dataframe_list

# 创建一个Excel写入对象
excel_writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')

# 遍历数据框列表，并将每个数据框写入Excel文件的不同sheet
for idx, df in enumerate(dataframe_list):
    sheet_name = f'sheet_{idx+1}'  # 生成sheet名称
    df.to_excel(excel_writer, sheet_name=sheet_name, index=False)  # 将数据框写入sheet

# 保存Excel文件
excel_writer.close()

技术指标，选出来的有：acd_df,br,bias6,bias12,bias24,cci,MAR10,MAR20,PSY,PVI,RC,ROC,RVI,SRMI_df,RSI,stoch_rsi

ACD、br、PVI衡量交易力量（多空力量），BIAS、cci是股价趋势回弹力量，MAR10、MAR20是均线价格比（运动趋势），PSY是心理预期（多空心理博弈），RC和ROC、RVI、SRMI_df是价格变化率指标，RSI、stoch_df是市场景气程度。

PVI是负的IC，其余是正的，因此PVI需要另行排序。

所以技术指标的，可以按照以上分类设置权重。

另外有还不错的IC的指标为：acd_df20, adtm_df，ar, aroon, cr, dbcd, j_values, MACD.

其中aroon是负的。

acd_df20、ar、aroon是多空力量指标，adtm_df、cr则是人气高低，dbcd比较像bias指标，j_values一般适合中短期趋势分析期货市场，MACD是均线指标，越大说明越要买。

基本面因子：TA_YOY、NPM_TTM、INVT、ROE_TTM、PTCF_Z、CSR、RAT、OP_QOQ、ROED、ROA_TTM、OT_Z。除了ROED，其余都是越大越好。



我建议的权重为基本面因子0.32，gpt评分因子0.08，技术面因子0.6。

其中基本面中TA_YOY占据0.06，NPM_TTM、INVT、ROE_TTM、PTCF_Z、CSR、RAT等权重占据0.18，OP_QOQ、ROED、ROA_TTM、OT_Z等权重占据0.08。
其中技术面中：acd_df,br,bias6,bias12,bias24,cci,MAR10,MAR20,PSY,PVI,RC,ROC,RVI,SRMI_df,RSI,stoch_rsi等权重占据0.56，acd_df20, adtm_df，ar, aroon, cr, dbcd, j_values, MACD等权重占据0.04

In [79]:
#录入gpt_score

# 读取Excel文件并设置为同名数据框
excel_file_path = 'gpt_score.xlsx'  # 替换为你的Excel文件路径
gpt_score = pd.read_excel(excel_file_path)
gpt_score.replace(0, np.nan, inplace=True)
gpt_score.replace([np.inf, -np.inf], np.nan, inplace=True)
gpt_score = standardize_dataframe(gpt_score)
gpt_score=gpt_score.fillna(0)
gpt_score=gpt_score.apply(lambda row: row.rank(pct=True), axis=1)

print(gpt_score)

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0        0.506      0.506      0.506      0.019      0.506      0.506   
1        0.512      0.512      0.512      0.031      0.512      0.512   
2        0.510      0.510      0.510      0.035      0.510      0.510   
3        0.507      0.507      0.507      0.039      0.507      0.507   
4        0.506      0.506      0.506      0.043      0.506      0.506   
..         ...        ...        ...        ...        ...        ...   
113      0.504      0.504      0.504      0.315      0.270      0.186   
114      0.504      0.504      0.504      0.315      0.260      0.180   
115      0.504      0.504      0.504      0.315      0.246      0.180   
116      0.500      0.500      0.500      0.311      0.246      0.180   
117      0.496      0.496      0.496      0.307      0.240      0.180   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0        0.506      0.506      0.506      0.5

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


基本面因子：TA_YOY、NPM_TTM、INVT、ROE_TTM、PTCF_Z、CSR、RAT、OP_QOQ、ROED、ROA_TTM、OT_Z。除了ROED，其余都是越大越好。

In [130]:
#录入基本面因子

f_path = 'fundamental_factors.xlsx'  # 替换为你的Excel文件路径
f_factors = pd.read_excel(f_path, sheet_name=None)

# 创建一个空字典，用于存储按照sheet name命名的数据框
dfs_by_sheet_name = {}

# 遍历字典，分别提取每个sheet，并将数据框命名为sheet的名字
for sheet_name, df in f_factors.items():
    # 将数据框存储在字典中，以sheet的名字作为键
    dfs_by_sheet_name[sheet_name] = df

TA_YOY=dfs_by_sheet_name["TA_YOY"]
print(TA_YOY)

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0      25.3737    13.9279    -7.7597     1.0484     6.4176     4.5915   
1      25.3737    13.9279    -7.7597     1.0484     6.4176     4.5915   
2      25.3737    13.9279    -7.7597     1.0484     6.4176     4.5915   
3      25.3737    13.9279    -7.7597     1.0484     6.4176     4.5915   
4      25.3737    13.9279    -7.7597     1.0484     6.4176     4.5915   
..         ...        ...        ...        ...        ...        ...   
113    10.1084    -2.1539    -6.3864    10.1552    20.3566    21.6529   
114    10.1084    -2.1539    -6.3864    10.1552    20.3566    21.6529   
115    10.1084    -2.1539    -6.3864    10.1552    20.3566    21.6529   
116    10.1084    -2.1539    -6.3864    10.1552    20.3566    21.6529   
117    10.1084    -2.1539    -6.3864    10.1552    20.3566    21.6529   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0       8.7026     2.0808  -6.388472     2.04

In [131]:
#因子有：TA_YOY、NPM_TTM、INVT、ROE_TTM、PTCF_Z、CSR、RAT、OP_QOQ、ROED、ROA_TTM、OT_Z
TA_YOY=dfs_by_sheet_name["TA_YOY"]
NPM_TTM=dfs_by_sheet_name["NPM_TTM"]
INVT=dfs_by_sheet_name["INVT"]
ROE_TTM=dfs_by_sheet_name["ROE_TTM"]
PTCF_Z=dfs_by_sheet_name["PTCF_Z"]
CSR=dfs_by_sheet_name["CSR"]
RAT=dfs_by_sheet_name["RAT"]
OP_QOQ=dfs_by_sheet_name["OP_QOQ"]
ROED=dfs_by_sheet_name["ROED"]
ROA_TTM=dfs_by_sheet_name["ROA_TTM"]
OT_Z=dfs_by_sheet_name["OT_Z"]
print(OT_Z)

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0     0.717984   0.717984  -0.257337   0.294047   0.717984   3.506032   
1     0.717984   0.717984  -0.257337   0.294047   0.717984   3.506032   
2     0.717984   0.717984  -0.257337   0.294047   0.717984   3.506032   
3     0.717984   0.717984  -0.257337   0.294047   0.717984   3.506032   
4     0.717984   0.717984  -0.257337   0.294047   0.717984   3.506032   
..         ...        ...        ...        ...        ...        ...   
113   0.871890   0.897150  -0.177220   0.871890   0.871890   0.871890   
114   0.871890   0.897150  -0.177220   0.871890   0.871890   0.871890   
115   0.871890   0.897150  -0.177220   0.871890   0.871890   0.871890   
116   0.871890   0.897150  -0.177220   0.871890   0.871890   0.871890   
117   0.871890   0.897150  -0.177220   0.871890   0.871890   0.871890   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...     688536.SH  \
0     0.130640   1.579399   2.451352  -0.078823  ... 

做策略。因子得分选股

其中基本面中TA_YOY占据0.06，NPM_TTM、INVT、ROE_TTM、PTCF_Z、CSR、RAT等权重占据0.18，OP_QOQ、ROED、ROA_TTM、OT_Z等权重占据0.08。
其中技术面中：acd_df,br,bias6,bias12,bias24,cci,MAR10,MAR20,PSY,PVI,RC,ROC,RVI,SRMI_df,RSI,stoch_rsi等权重占据0.56，acd_df20, adtm_df，ar, aroon, cr, dbcd, j_values, MACD等权重占据0.04

In [146]:
#已有标准化的函数standardize_dataframe(input_df)，以及将缺失值设为0的方式df.fillna(0)

#对技术面因子处理。

chosen_factors=[acd_df,br,bias6,bias12,bias24,cci,MAR10,MAR20,PSY,PVI,RC,ROC,RVI,SRMI_df,RSI,stoch_rsi,acd_df20, adtm_df,ar, aroon, cr, dbcd, j_values, MACD]
processed_factors=[]
#对每一个factor df做标准化和缺失值填补。并取得百分数排序。
for df in chosen_factors:
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = standardize_dataframe(df)
    df=df.fillna(0)
    df=df.apply(lambda row: row.rank(pct=True), axis=1)
    df = df.tail(118)
    df.reset_index(drop=True, inplace=True)
    processed_factors.append(df)

print(processed_factors[1])



c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in div

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0        0.712      0.318      0.116      0.092      0.392      0.194   
1        0.704      0.374      0.112      0.164      0.212      0.172   
2        0.656      0.372      0.166      0.216      0.096      0.244   
3        0.604      0.408      0.242      0.202      0.102      0.388   
4        0.598      0.452      0.224      0.268      0.140      0.520   
..         ...        ...        ...        ...        ...        ...   
113      0.356      0.316      0.144      0.958      0.742      0.826   
114      0.470      0.166      0.076      0.950      0.522      0.804   
115      0.424      0.162      0.046      0.946      0.750      0.796   
116      0.412      0.142      0.050      0.974      0.826      0.028   
117      0.356      0.150      0.058      0.978      0.890      0.032   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0        0.102      0.044      0.386      0.0

In [147]:
print(len(processed_factors))

24


In [148]:
#处理基本面因子
chosen_f_factors=[TA_YOY,NPM_TTM,INVT,ROE_TTM,PTCF_Z,CSR,RAT,OP_QOQ,ROED,ROA_TTM,OT_Z]
processed_f_fatcors=[]
for df in chosen_f_factors:
    df=df.apply(lambda row: row.rank(pct=True), axis=1)
    processed_f_fatcors.append(df)
print(len(processed_f_fatcors))
print(processed_f_fatcors[1])


11
     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0        0.120      0.360      0.124      0.108      0.506      0.388   
1        0.120      0.360      0.124      0.108      0.506      0.388   
2        0.120      0.360      0.124      0.108      0.506      0.388   
3        0.120      0.360      0.124      0.108      0.506      0.388   
4        0.120      0.360      0.124      0.108      0.506      0.388   
..         ...        ...        ...        ...        ...        ...   
113      0.202      0.304      0.256      0.280      0.574      0.804   
114      0.202      0.304      0.256      0.280      0.574      0.804   
115      0.202      0.304      0.256      0.280      0.574      0.804   
116      0.202      0.304      0.256      0.280      0.574      0.804   
117      0.202      0.304      0.256      0.280      0.574      0.804   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0        0.282      0.316      0.290      

In [149]:
final_factors=[]
for df in processed_factors:
    final_factors.append(df)
print(len(final_factors))

24


In [150]:
for df in processed_f_fatcors:
    final_factors.append(df)
print(len(final_factors))

35


In [151]:
final_factors.append(gpt_score)
print(len(final_factors))

36


In [152]:
#处理两个越小越好的数据框
final_factors[9]=1-final_factors[9]#PVI
print(final_factors[9])

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0        0.666      0.672      0.272      0.260      0.316      0.932   
1        0.622      0.832      0.260      0.254      0.290      0.250   
2        0.690      0.644      0.658      0.922      0.822      0.770   
3        0.718      0.684      0.678      0.934      0.846      0.786   
4        0.874      0.662      0.262      0.974      0.304      0.910   
..         ...        ...        ...        ...        ...        ...   
113      0.412      0.618      0.152      0.960      0.212      0.890   
114      0.948      0.754      0.184      0.288      0.280      0.270   
115      0.360      0.562      0.862      0.188      0.186      0.176   
116      0.358      0.414      0.078      0.950      0.898      0.844   
117      0.674      0.778      0.196      0.270      0.274      0.260   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0        0.312      0.674      0.610      0.0

In [153]:
final_factors[19]=1-final_factors[19]#aroon
print(final_factors[19])

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0        0.526      0.542      0.228      0.450      0.300      0.468   
1        0.262      0.664      0.230      0.390      0.226      0.506   
2        0.226      0.600      0.398      0.318      0.204      0.586   
3        0.380      0.558      0.346      0.246      0.364      0.538   
4        0.354      0.528      0.318      0.220      0.336      0.870   
..         ...        ...        ...        ...        ...        ...   
113      0.846      0.486      0.288      0.734      0.740      0.746   
114      0.816      0.124      0.260      0.708      0.712      0.718   
115      0.772      0.104      0.210      0.838      0.662      0.762   
116      0.760      0.064      0.192      0.824      0.644      0.268   
117      0.666      0.058      0.140      0.812      0.576      0.208   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0        0.162      0.512      0.262      0.4

In [154]:
final_factors[32]=1-final_factors[32]#ROED
print(final_factors[32])

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0        0.592      0.068      0.766      0.942       0.29      0.786   
1        0.592      0.068      0.766      0.942       0.29      0.786   
2        0.592      0.068      0.766      0.942       0.29      0.786   
3        0.592      0.068      0.766      0.942       0.29      0.786   
4        0.592      0.068      0.766      0.942       0.29      0.786   
..         ...        ...        ...        ...        ...        ...   
113      0.834      0.806      0.648      0.876       0.11      0.606   
114      0.834      0.806      0.648      0.876       0.11      0.606   
115      0.834      0.806      0.648      0.876       0.11      0.606   
116      0.834      0.806      0.648      0.876       0.11      0.606   
117      0.834      0.806      0.648      0.876       0.11      0.606   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0        0.686      0.570      0.030      0.7

其中基本面中TA_YOY占据0.06，NPM_TTM、INVT、ROE_TTM、PTCF_Z、CSR、RAT等权重占据0.18，OP_QOQ、ROED、ROA_TTM、OT_Z等权重占据0.08。
其中技术面中：acd_df,br,bias6,bias12,bias24,cci,MAR10,MAR20,PSY,PVI,RC,ROC,RVI,SRMI_df,RSI,stoch_rsi等权重占据0.56，acd_df20, adtm_df，ar, aroon, cr, dbcd, j_values, MACD等权重占据0.04

In [155]:
print(len(final_factors))

36


In [156]:
#假设weights是给定的权重列表
weights=[0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.035,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,
         0.06,0.03,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.02,0.02,0.08]
print(len(weights))

36


In [157]:
# 计算综合得分
composite_scores = sum(factor * weight for factor, weight in zip(final_factors, weights))
#得到股票根据综合排名后每日的得分。（百分位数）
stock=composite_scores.apply(lambda row: row.rank(pct=True), axis=1)
print(stock)

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0        0.412      0.396      0.032      0.108      0.218      0.646   
1        0.324      0.612      0.032      0.068      0.286      0.420   
2        0.360      0.788      0.090      0.144      0.410      0.536   
3        0.610      0.766      0.622      0.116      0.390      0.886   
4        0.782      0.774      0.400      0.302      0.310      0.964   
..         ...        ...        ...        ...        ...        ...   
113      0.406      0.550      0.126      0.856      0.834      0.966   
114      0.784      0.576      0.070      0.844      0.780      0.910   
115      0.528      0.054      0.168      0.782      0.682      0.862   
116      0.432      0.038      0.092      0.860      0.696      0.780   
117      0.176      0.050      0.054      0.748      0.622      0.094   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0        0.078      0.382      0.050      0.3

In [188]:
#得到每只股票的综合得分后，就可以假定我们每日清仓卖出一只股票，然后全仓买入另一只当天得分较高的股票，然后就以下一日的回报率作为我们买了这支股票的回报率。
# 找出每天综合得分最高的股票的位置
selected_stocks = composite_scores.idxmax(axis=1)

print(selected_stocks)

0      002465.SZ
1      000975.SZ
2      000975.SZ
3      600369.SH
4      600369.SH
         ...    
113    300012.SZ
114    300012.SZ
115    300012.SZ
116    002078.SZ
117    002078.SZ
Length: 118, dtype: object


In [162]:
tplus2return = logreturnnext.iloc[1:]

data = [[0] * 500]
df = pd.DataFrame(data, columns=tplus2return.columns)

# 使用pd.concat()函数将两个数据框按行连接
tplus2return = pd.concat([tplus2return, df])
tplus2return=tplus2return.tail(118)
tplus2return.reset_index(drop=True, inplace=True)
print(tplus2return)

     000009.SZ  000012.SZ  000021.SZ  000027.SZ  000031.SZ  000039.SZ  \
0     0.030200   0.022196   0.024520   0.013501   0.026397   0.019654   
1     0.018141   0.007290   0.042343   0.001914   0.005994   0.031024   
2     0.015608   0.000000   0.000000   0.007619  -0.003992   0.005376   
3    -0.008889   0.000000   0.006612   0.000000   0.007968  -0.001789   
4    -0.006719   0.004831   0.006568   0.003788  -0.009970   0.006247   
..         ...        ...        ...        ...        ...        ...   
113  -0.023811   0.009501   0.002407   0.000000  -0.004626  -0.001489   
114  -0.001723  -0.004739   0.007273   0.006410   0.001544   0.000745   
115  -0.015639  -0.011948  -0.018282  -0.009631  -0.009302  -0.004671   
116   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
117   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   

     000050.SZ  000060.SZ  000066.SZ  000089.SZ  ...  688536.SH  688538.SH  \
0     0.016772   0.024939   0.034947   0.0156

In [190]:
selected_returns = tplus2return.apply(lambda row: row[selected_stocks[row.name]], axis=1)
print(selected_returns)

0      0.013374
1     -0.014375
2     -0.041385
3     -0.009259
4     -0.059898
         ...   
113    0.001876
114    0.010257
115    0.001854
116    0.000000
117    0.000000
Length: 118, dtype: float64


In [191]:
#所用数据即，通过每一天通过综合得分最高的股票，来找到这只股票下一天的收益率，作为我们的收益率，最后可以算年化收益率。至于风险，则可以计算收益率的年化标准差作为风险。
test_returns=selected_returns[0:116] #即2019年上半年的收益率
print(test_returns)


0      0.013374
1     -0.014375
2     -0.041385
3     -0.009259
4     -0.059898
         ...   
111    0.000000
112    0.010382
113    0.001876
114    0.010257
115    0.001854
Length: 116, dtype: float64


In [178]:
# 将一维数组转换为DataFrame
df = pd.DataFrame(test_returns, columns=['returns'])  # 列名可根据实际情况修改

# 将DataFrame保存为Excel文件
excel_file_path = 'rt.xlsx'  # 指定保存的Excel文件名及路径
df.to_excel(excel_file_path, index=False)  # index=False表示不保存索引列

In [194]:
#只考虑gpt因子，进行选择。
selected_gpt = gpt_score.idxmax(axis=1)
gpt_returns = tplus2return.apply(lambda row: row[selected_gpt[row.name]], axis=1)
print(gpt_returns)

0      0.032857
1      0.020001
2     -0.017890
3      0.013986
4      0.020001
         ...   
113   -0.005362
114         NaN
115         NaN
116    0.000000
117    0.000000
Length: 118, dtype: float64


In [193]:
# 将一维数组转换为DataFrame
df = pd.DataFrame(gpt_returns, columns=['returns'])  # 列名可根据实际情况修改

# 将DataFrame保存为Excel文件
excel_file_path = 'rt1.xlsx'  # 指定保存的Excel文件名及路径
df.to_excel(excel_file_path, index=False)  # index=False表示不保存索引列


In [197]:
#只考虑技术因子进行选择
print(len(processed_factors))
weights=[0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.025,0.025,0.025,0.025,0.025,0.025,0.025,0.025]
print(len(weights))

composite_scores = sum(factor * weight for factor, weight in zip(processed_factors, weights))
selected_tech = composite_scores.idxmax(axis=1)
tech_returns = tplus2return.apply(lambda row: row[selected_tech[row.name]], axis=1)
print(tech_returns)

24
24
0      0.003688
1      0.037286
2     -0.015466
3     -0.001248
4     -0.067315
         ...   
113    0.001876
114    0.010257
115   -0.019223
116    0.000000
117    0.000000
Length: 118, dtype: float64


In [198]:
# 将一维数组转换为DataFrame
df = pd.DataFrame(tech_returns, columns=['returns'])  # 列名可根据实际情况修改

# 将DataFrame保存为Excel文件
excel_file_path = 'rt2.xlsx'  # 指定保存的Excel文件名及路径
df.to_excel(excel_file_path, index=False)  # index=False表示不保存索引列

结果并不好，说明可能需要重新挑选。

技术指标，选出来的有：acd_df,br,bias6,bias12,bias24,cci,MAR10,MAR20,PSY,PVI,RC,ROC,RVI,SRMI_df,RSI,stoch_rsi

ACD、br、PVI衡量交易力量（多空力量），BIAS、cci是股价趋势回弹力量，MAR10、MAR20是均线价格比（运动趋势），PSY是心理预期（多空心理博弈），RC和ROC、RVI、SRMI_df是价格变化率指标，RSI、stoch_df是市场景气程度。

PVI是负的IC，其余是正的，因此PVI需要另行排序。

所以技术指标的，可以按照以上分类设置权重。

另外有还不错的IC的指标为：acd_df20, adtm_df，ar, aroon, cr, dbcd, j_values, MACD.

其中aroon是负的。

acd_df20、ar、aroon是多空力量指标，adtm_df、cr则是人气高低，dbcd比较像bias指标，j_values一般适合中短期趋势分析期货市场，MACD是均线指标，越大说明越要买。

以上是原有的技术指标。

adtm（人气度）、cr（行情）、j_values（行情）、PSY（心理博弈）、PVI（市场行情）、RC（市场活跃）、RVI（市场趋势）、SRMI_df（市场趋势）、stoch_rsi（市场走势）、asi（市场趋势）可以保留（但权重要小一些），添加bbi指标（负的），bias6、bias12、bias24、cci、MAR10、MAR20、MACD、ROC、RSI指标保留

bias6,bias12,bias24,cci,MAR10,MAR20,MACD,ROC,RSI,adtm,cr,j_values,PSY,PVI,RC,RVI,SRMI_df,stoch_rsi,asi,bbi  最后保留的因子

In [201]:
#对技术面因子处理。

chosen_tech=[bias6,bias12,bias24,cci,MAR10,MAR20,MACD,ROC,RSI,adtm_df,cr,j_values,PSY,PVI,RC,RVI,SRMI_df,stoch_rsi,asi,bbi]
processed_tech=[]
#对每一个factor df做标准化和缺失值填补。并取得百分数排序。
for df in chosen_tech:
    df = standardize_dataframe(df)
    df=df.fillna(0)
    df=df.apply(lambda row: row.rank(pct=True), axis=1)
    df = df.tail(118)
    df.reset_index(drop=True, inplace=True)
    processed_tech.append(df)

processed_tech[19]=1-processed_tech[19]
print(len(processed_tech))
print(len(processed_tech))
weights=[0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08]
print(len(weights))

composite_scores = sum(factor * weight for factor, weight in zip(processed_tech, weights))
selected_tech = composite_scores.idxmax(axis=1)
tech_returns = tplus2return.apply(lambda row: row[selected_tech[row.name]], axis=1)
print(tech_returns)

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in div

20
20
20
0     -0.007273
1     -0.033398
2      0.003766
3     -0.004488
4     -0.059898
         ...   
113    0.001876
114    0.010257
115    0.001854
116    0.000000
117    0.000000
Length: 118, dtype: float64


In [202]:
# 将一维数组转换为DataFrame
df = pd.DataFrame(tech_returns, columns=['returns'])  # 列名可根据实际情况修改

# 将DataFrame保存为Excel文件
excel_file_path = 'rt2.xlsx'  # 指定保存的Excel文件名及路径
df.to_excel(excel_file_path, index=False)  # index=False表示不保存索引列

阻止了累计为负收益率。所以接下来是继续挑选因子使得结果更好。

adtm（人气度）、cr（行情）、j_values（行情）、PSY（心理博弈）、PVI（市场行情）、RC（市场活跃）、RVI（市场趋势）、SRMI_df（市场趋势）、stoch_rsi（市场走势）、asi（市场趋势）可以保留（但权重要小一些），添加bbi指标（负的），bias6、bias12、bias24、cci、MAR10、MAR20、MACD、ROC、RSI指标保留

以上是刚刚挑选的因子

重新挑选：bias6、bias12、bias24、cci、MAR10、MAR20、MACD、ROC、RSI指标保留

In [204]:
#对技术面因子处理。

chosen_tech=[bias6,bias12,bias24,cci,MAR10,MAR20,MACD,ROC,RSI]
processed_tech=[]
#对每一个factor df做标准化和缺失值填补。并取得百分数排序。
for df in chosen_tech:
    df = standardize_dataframe(df)
    df=df.fillna(0)
    df=df.apply(lambda row: row.rank(pct=True), axis=1)
    df = df.tail(118)
    df.reset_index(drop=True, inplace=True)
    processed_tech.append(df)

print(len(processed_tech))
print(len(processed_tech))
weights=[0.12,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11]
print(len(weights))

composite_scores = sum(factor * weight for factor, weight in zip(processed_tech, weights))
selected_tech = composite_scores.idxmax(axis=1)
tech_returns = tplus2return.apply(lambda row: row[selected_tech[row.name]], axis=1)
print(tech_returns)


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in div

9
9
9
0     -0.007273
1     -0.033398
2     -0.017922
3      0.000000
4     -0.067315
         ...   
113    0.001283
114   -0.035229
115   -0.032035
116    0.000000
117    0.000000
Length: 118, dtype: float64


In [205]:
# 将一维数组转换为DataFrame
df = pd.DataFrame(tech_returns, columns=['returns'])  # 列名可根据实际情况修改

# 将DataFrame保存为Excel文件
excel_file_path = 'rt2.xlsx'  # 指定保存的Excel文件名及路径
df.to_excel(excel_file_path, index=False)  # index=False表示不保存索引列

现在技术指标基本OK，能够获得很大的收益率。

测验基本面指标。

In [209]:
#处理基本面因子
chosen_f_factors=[TA_YOY,NPM_TTM,INVT,ROE_TTM,PTCF_Z,CSR,RAT,OP_QOQ,ROA_TTM,OT_Z]
processed_f_fatcors=[]
for df in chosen_f_factors:
    df=df.apply(lambda row: row.rank(pct=True), axis=1)
    processed_f_fatcors.append(df)
print(len(processed_f_fatcors))

weights=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
print(len(weights))

composite_scores = sum(factor * weight for factor, weight in zip(processed_f_fatcors, weights))
selected_f = composite_scores.idxmax(axis=1)
f_returns = tplus2return.apply(lambda row: row[selected_f[row.name]], axis=1)
print(f_returns)

10
10
0      0.086204
1     -0.034330
2     -0.026072
3     -0.003781
4     -0.012387
         ...   
113    0.019487
114    0.018253
115   -0.003451
116    0.000000
117    0.000000
Length: 118, dtype: float64


In [210]:
# 将一维数组转换为DataFrame
df = pd.DataFrame(f_returns, columns=['returns'])  # 列名可根据实际情况修改

# 将DataFrame保存为Excel文件
excel_file_path = 'rt2.xlsx'  # 指定保存的Excel文件名及路径
df.to_excel(excel_file_path, index=False)  # index=False表示不保存索引列

基本面因子，基本ok，收益率偏低。但合乎选择。

In [211]:
#已有processed_tech  9个指标
final_factors=[]
for df in processed_tech:
    final_factors.append(df)
print(len(final_factors))



9


In [212]:
for df in processed_f_fatcors:
    final_factors.append(df)
print(len(final_factors))

19


In [213]:
final_factors.append(gpt_score)
print(len(final_factors))

20


现在是9个技术指标，10个基本面指标，1个gpt评分。

由于技术指标能带来更高收益率，gpt评分次之，然后才是基本面，因此暂时选择技术指标共占据0.72，gpt评分占据0.18，基本面占据0.3

准确的说，是需要不断的调整权重的。

In [239]:
weights=[0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05]
print(len(weights))
composite_scores = sum(factor * weight for factor, weight in zip(final_factors, weights))
selected_final = composite_scores.idxmax(axis=1)
final_returns = tplus2return.apply(lambda row: row[selected_final[row.name]], axis=1)
print(final_returns)

20
0      0.006683
1     -0.014375
2     -0.026072
3     -0.013529
4     -0.009122
         ...   
113    0.003063
114    0.011151
115   -0.003451
116    0.000000
117    0.000000
Length: 118, dtype: float64


In [240]:
# 将一维数组转换为DataFrame
df = pd.DataFrame(final_returns, columns=['returns'])  # 列名可根据实际情况修改

# 将DataFrame保存为Excel文件
excel_file_path = 'rt2.xlsx'  # 指定保存的Excel文件名及路径
df.to_excel(excel_file_path, index=False)  # index=False表示不保存索引列

基本上完成，对于策略的调整无非是在因子的选择上和权重的确定进行改进。